## **Agents in LangChain**

We'll be working with agents and tools:
- **Agents**: Autonomous systems that make decisions and take actions
- **Tools**: Functions that agents can use to perform specific tasks, such as
  - Data query
  - Research reports
  - Data analysis

Our agents will use tools to perform the tasks: 
- Solve math problems,
- Search Wikipedia,
- Determine when to swicth between tools and LLMs based on a given task.

Combining tools with agents can also improve accuracy in domains like coding and math. <br>
LangChain uses specific tools to break problems into smaller steps, reducing errors. For example, we can use a tool to handle to Order Of Operations in math.

### **Expanding agents with LangGraph**

LangGraph an enhance tool use even further by structuring tasks in workflows called graphs.



<div style="display: flex;">
    <!-- Left Column -->
    <div style="width: 25%; padding: 10px;">
    In these graphs, tasks called "<b>nodes</b>" are connected by rules called "<b>edges</b>". For example, a database query node can link to a document retrieval node, with an edge pointing to which document is retrieved.
    </div>
    <!-- Right Column -->
    <div style="width: 48%; padding: 10px;">
        <img src='./images/graph-str.png' width=25%>
    </div>
</div>

### **Create a ReAct agent**

Now, let's create a basic ReAct agent that does math.

- The `tool` module imported from `langchain_core.tools` lets us use custom functions. 
- `ChatOpenAI` imported from `langchain_openai` enables communication with OpenAI's language models.
- The `create_react_agent` module, imported from `langgraph.prebuilt` functions, helps us create a ReAct agent that can reason and use tools. 
- Finally, the `math` module lets us perform standard math. 

In [1]:
from IPython.display import display, Markdown  # For rendering Markdown text in Jupyter Notebooks
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
import math

Then, once our model has been defined, we'll create a ReAct agent by passing the model and a pre-defined tool to the create_react_agent() function. 

In [2]:
import os       # Import the os module to access environment variables for the OpenAI API key
api_key = os.environ["OPENAI_API_KEY"]

# Pre-defined tool
@tool
def evaluate_expression(expression: str) -> float:
    """Evaluates a mathematical expression given as a string."""
    try:
        return eval(expression)  # Use eval carefully, only for safe math expressions
    except Exception as e:
        return f"Error: {str(e)}"

# List of tools
tools = [evaluate_expression]

model = ChatOpenAI(api_key=api_key, model = "gpt-4o-mini")

# Create the agent
agent = create_react_agent(model, tools)

Since our tool is a basic calculator, we'll define a math query as a string. We'll pass this query, labeled `"human"`, to the agent using the `.invoke()` method, storing the output as a response. We'll then use the `.content` attribute to print the last item in `"messages"` of `"response"` to get the agent's answer. 

In [3]:
# Create a query
query = "What is (2+8) multiplied by 9?"

# Invoke the agent and print the response
response = agent.invoke({"messages": [("human", query)]})

print(response['messages'][-1].content)

The result of (2 + 8) multiplied by 9 is 90.


### **EXAMPLE**

In [4]:
from langchain_core.tools import tool

@tool
def count_r_in_word(word: str) -> str:
    """Counts the number of 'r's in a given word and returns a formatted response."""
    count = word.lower().count('r')
    return f"The word \"{word}\" contains {count} 'r's"

In [5]:
# Create the agent
app = create_react_agent(model=model, tools=[count_r_in_word])

# Create a query
query = "How many r's are in the word 'Strawberry'?"

# Invoke the agent and store the response
response = app.invoke({"messages": [("human", query)]})

# Print the agent's response
print(response['messages'][-1].content)

The word "Strawberry" contains 3 'r's.


## **Building custom tools**

1. We'll start by using a _decorator_ called $ \text{\textcolor{green}{@tool}} $ that LangChain uses to recognize custom functions as tools. 

2. After the decorator, we'll create a function called `rectangle_area` that takes in a _string_ as an input. 

3. We then include a _docstring_ to describe the function's purpose, specifying that it calculates the area of a rectangle given the lengths of two sides, `a` and `b`. 

4. Inside the function, we `split` the input string extracted from the query into the two values representing sides a and b. 

5. To multiply the sides, we _strip_ any whitespace around the string inputs using Python's `.strip()` method and then convert each string to a float. 

6. We then multiply sides a and b together to calculate and return the area of the rectangle.

In [6]:
@tool
def rectangle_area(input: str) -> float:
    """Calculate the area of a rectangle given its lengths of sides a and b."""
    sides = input.split(',')
    a = float(sides[0].strip())
    b = float(sides[1].strip())
    return a * b

Now that we have our tool, let's make sure LangChain can access it by passing it within a list called `"tools"`. Although we're only using one tool here, it's possible to list more, depending on our workflow. Then, we'll create a variable called `"query"` that accepts a question from the user in the form of natural language. 

We'll then create our ReAct agent called `app`, passing in the model, and the tool we just built.

To test that the agent works, we'll invoke the agent we just created, passing in the `query` we defined and then print the agent's `response` by identifying the last item in `"messages"` using the `.content` attribute.

In [7]:
# Define the tools that the agent will use
tools = [rectangle_area]

# Create a query using natural language
query="What is the area of a rectangle with sides 5 and 7?"

# Pass in the tool and invoke the agent
app = create_react_agent(model, tools)

# Invoke the agent and print the response
response = app.invoke({"messages": [("human", query)]})

print(response['messages'][-1].content)

The area of the rectangle with sides 5 and 7 is 35 square units.


LangChain also has an extensive library of pre-built tools for solving many other problems such as 
- database querying, 
- web scraping, and 
- image generation, 

which can be incorporated by referencing [LangChain's API guide](https://python.langchain.com/docs/integrations/tools/). We can also reference [LangChain's Custom tool guide](https://python.langchain.com/docs/how_to/custom_tools/) for using tool decorators to build other custom tools!

## **Conversation with a ReAct agent**

So far, we've just been printing the agent's outputs. It's also useful to know that our agent is responding correctly by printing both the user's query as well as the response.

### **Conversation history**

To set up our conversation history, we'll import the `HumanMessage` and `AIMessage` modules from `langchain_core.messages`.

Then, wel'll set up a variable called `message_history` that will store all of our messages.

Next, we'll defina new query that will ask a new question without providing any additional contextual information. 
(Here, we want know the area if a new rectangle with different dimensions)

Next,  we'll invoke the app object again, this time passing both message histpry anf new query to the agent within a dictionary.

We'll then filter out only the relevant messages from the agent's response. Here, we'll use a list comprehension to select both HumanMessage and AIMessage instances that contain actual content. When applied to `msg.content`, the `.strip()` method removes any trailing whitespaces.

Finally, we'll format and print the conversation extracted from `msg.content`, with each message labeled using its proper class name. The `"user_input"` is our new query, while the `"agent_output"` will print the full conversation and repeat the agent's most recent output, which uis useful for debugging.

Let's see if we can produce the elements we need:

In [13]:
from langchain_core.messages import HumanMessage, AIMessage

message_history = response["messages"]

new_query = "What about one with sides 4 and 3?"

# Invoke the app with the full message history
messages = app.invoke({"messages": message_history + [("human", new_query)]})

# Extract the human and AI messages
filtered_messages = [msg for msg in messages["messages"] if isinstance(msg,(HumanMessage, AIMessage)) and msg.content.strip()]

# Format and print the final result
print({
    "user_input": new_query,
    "agent_output": [f"{msg.__class__.__name__}: {msg.content}" for msg in filtered_messages]
})


{'user_input': 'What about one with sides 4 and 3?', 'agent_output': ['HumanMessage: What is the area of a rectangle with sides 5 and 7?', 'AIMessage: The area of the rectangle with sides 5 and 7 is 35 square units.', 'HumanMessage: What about one with sides 4 and 3?', 'AIMessage: The area of the rectangle with sides 4 and 3 is 12 square units.']}


We have our new query labeled `"user_input"`. Then, we have our `agent_output` listing the full conversation with labeled human and AI messages. 

When we ask the agent to list the last message, we have our most recent query and answer. Everything is in good shape!

In [9]:
# Replace the existing print block at the end with this:
print({
    "user_input": new_query,
    "agent_output": f"{messages['messages'][-1].__class__.__name__}: {messages['messages'][-1].content}"
})

{'user_input': 'What about one with sides 4 and 3?', 'agent_output': 'AIMessage: The area of the rectangle with sides 4 and 3 is 12 square units.'}


Full functional code is below.

In [ ]:
from IPython.display import display, Markdown
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
import math
import os 
from langchain_core.messages import HumanMessage, AIMessage

api_key = os.environ["OPENAI_API_KEY"]

@tool
def rectangle_area(input: str) -> float:
    """Calculate the area of a rectangle given its lengths of sides a and b."""
    sides = input.split(',')
    a = float(sides[0].strip())
    b = float(sides[1].strip())
    return a * b

model = ChatOpenAI(api_key=api_key, model = "gpt-4o-mini")

# Define the tools that the agent will use
tools = [rectangle_area]

# Create a query using natural language
query = "What is the area of a rectangle with sides 5 and 7?"

# Pass in the tool and invoke the agent
app = create_react_agent(model, tools)

# Invoke the agent and print the response
response = app.invoke({"messages": [("human", query)]})

# Store message history from the first conversation
message_history = response["messages"]

# Create a new query
new_query = "What about one with sides 4 and 3?"

# Invoke the app with the full message history
messages = app.invoke({"messages": message_history + [("human", new_query)]})

# Extract the human and AI messages
filtered_messages = [msg for msg in messages["messages"] if isinstance(msg,(HumanMessage, AIMessage)) and msg.content.strip()]

# Format and print the final result
print({
    "user_input": new_query,
    "agent_output": [f"{msg.__class__.__name__}: {msg.content}" for msg in filtered_messages]
})

{'user_input': 'What about one with sides 4 and 3?', 'agent_output': ['HumanMessage: What is the area of a rectangle with sides 5 and 7?', 'AIMessage: The area of the rectangle with sides 5 and 7 is 35 square units.', 'HumanMessage: What about one with sides 4 and 3?', 'AIMessage: The area of the rectangle with sides 4 and 3 is 12 square units.']}
